# MVP-V: 객체 추적

이 노트북은 ByteTrack 또는 DeepSORT를 사용하여 프레임 간 객체 추적을 수행합니다.

## 목표
- ByteTrack 또는 DeepSORT 구현
- 프레임 간 객체 ID 유지
- 속도 추정 (프레임 간 위치 변화)

In [ ]:
# Cell 1: 라이브러리 설치 및 임포트
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from collections import defaultdict
from scipy.spatial.distance import iou

# ByteTrack 사용 시 설치 필요: pip install bytetrack
# 또는 직접 구현

print("추적 모듈 준비 완료")

In [ ]:
# Cell 2: 간단한 Kalman Filter 기반 추적 구현
class SimpleTracker:
    """
    간단한 Kalman Filter 기반 객체 추적기
    """
    def __init__(self, max_age=30, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.tracks = []
        self.frame_count = 0
        self.next_id = 1
    
    def update(self, detections):
        """
        새로운 탐지 결과로 추적 업데이트
        detections: list of [x1, y1, x2, y2, conf, class] 또는 [center_x, center_y, w, h, conf, class]
        """
        self.frame_count += 1
        
        # 기존 트랙 업데이트
        matched, unmatched_tracks, unmatched_detections = self._associate_detections_to_trackers(detections)
        
        # 매칭된 트랙 업데이트
        for track_idx, det_idx in matched:
            self.tracks[track_idx].update(detections[det_idx])
            self.tracks[track_idx].time_since_update = 0
        
        # 매칭되지 않은 탐지에 대해 새 트랙 생성
        for det_idx in unmatched_detections:
            self._create_track(detections[det_idx])
        
        # 오래된 트랙 제거
        self.tracks = [t for t in self.tracks if t.time_since_update < self.max_age]
        
        return self.tracks
    
    def _associate_detections_to_trackers(self, detections):
        """탐지와 트랙 매칭 (IoU 기반)"""
        if len(self.tracks) == 0:
            return [], [], list(range(len(detections)))
        
        # IoU 행렬 계산
        iou_matrix = np.zeros((len(self.tracks), len(detections)))
        for t, track in enumerate(self.tracks):
            for d, det in enumerate(detections):
                iou_matrix[t, d] = self._compute_iou(track.get_state(), det)
        
        # 간단한 헝가리안 알고리즘 또는 greedy matching
        matched = []
        unmatched_tracks = list(range(len(self.tracks)))
        unmatched_detections = list(range(len(detections)))
        
        # 최대 IoU 매칭
        while True:
            if len(unmatched_tracks) == 0 or len(unmatched_detections) == 0:
                break
            
            max_iou = 0
            best_match = None
            
            for t in unmatched_tracks:
                for d in unmatched_detections:
                    if iou_matrix[t, d] > max_iou and iou_matrix[t, d] > self.iou_threshold:
                        max_iou = iou_matrix[t, d]
                        best_match = (t, d)
            
            if best_match is None:
                break
            
            matched.append(best_match)
            unmatched_tracks.remove(best_match[0])
            unmatched_detections.remove(best_match[1])
        
        return matched, unmatched_tracks, unmatched_detections
    
    def _compute_iou(self, box1, box2):
        """IoU 계산"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 < x1 or y2 < y1:
            return 0.0
        
        inter = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - inter
        
        return inter / union if union > 0 else 0.0
    
    def _create_track(self, detection):
        """새 트랙 생성"""
        track = Track(detection, self.next_id, self.frame_count)
        self.tracks.append(track)
        self.next_id += 1


class Track:
    """단일 객체 트랙"""
    def __init__(self, detection, track_id, frame_id):
        self.track_id = track_id
        self.detections = [detection]
        self.frame_ids = [frame_id]
        self.time_since_update = 0
        self.hit_streak = 1
    
    def update(self, detection):
        """트랙 업데이트"""
        self.detections.append(detection)
        self.time_since_update = 0
        self.hit_streak += 1
    
    def get_state(self):
        """현재 상태 반환 (마지막 탐지)"""
        return self.detections[-1][:4]  # x1, y1, x2, y2


print("추적기 클래스 정의 완료")

In [ ]:
# Cell 3: 추적 파이프라인 (YOLO 탐지 결과 사용)
def track_objects(tracker, yolo_results, frame_idx):
    """
    YOLO 탐지 결과를 추적기에 입력
    """
    # YOLO 결과를 [x1, y1, x2, y2, conf, class_id] 형식으로 변환
    detections = []
    for box in yolo_results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])
        class_id = int(box.cls[0])
        detections.append([x1, y1, x2, y2, conf, class_id])
    
    # 추적 업데이트
    tracks = tracker.update(detections)
    
    return tracks


# 테스트를 위한 추적기 초기화
tracker = SimpleTracker(max_age=30, min_hits=3, iou_threshold=0.3)
print("추적기 초기화 완료")

In [ ]:
# Cell 4: 속도 추정 함수
def estimate_velocity(track, fps=10, pixel_to_meter=0.1):
    """
    프레임 간 위치 변화로부터 속도 추정
    
    Args:
        track: Track 객체
        fps: 초당 프레임 수
        pixel_to_meter: 픽셀을 미터로 변환하는 비율 (칼리브레이션 필요)
    
    Returns:
        velocity_x, velocity_y: x, y 방향 속도 (m/s)
    """
    if len(track.detections) < 2:
        return 0, 0
    
    # 최근 2개 탐지의 중심점 계산
    det1 = track.detections[-2]
    det2 = track.detections[-1]
    
    center1_x = (det1[0] + det1[2]) / 2
    center1_y = (det1[1] + det1[3]) / 2
    center2_x = (det2[0] + det2[2]) / 2
    center2_y = (det2[1] + det2[3]) / 2
    
    # 픽셀 단위 이동
    dx = (center2_x - center1_x) * pixel_to_meter
    dy = (center2_y - center1_y) * pixel_to_meter
    
    # 시간 차이
    dt = 1.0 / fps
    
    # 속도 (m/s)
    velocity_x = dx / dt
    velocity_y = dy / dt
    
    return velocity_x, velocity_y

print("속도 추정 함수 정의 완료")

In [ ]:
# Cell 5: 추적 결과 시각화
def visualize_tracking(image, tracks, show_velocity=False):
    """
    추적 결과를 이미지에 시각화
    """
    fig, ax = plt.subplots(1, 1, figsize=(15, 8))
    ax.imshow(image)
    ax.set_title("Object Tracking Results", fontsize=14)
    ax.axis('off')
    
    # 트랙별 색상 할당
    colors = plt.cm.tab20(np.linspace(0, 1, len(tracks)))
    
    for i, track in enumerate(tracks):
        if len(track.detections) == 0:
            continue
        
        det = track.detections[-1]
        x1, y1, x2, y2 = det[:4]
        
        # 바운딩 박스 그리기
        color = colors[i % len(colors)]
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                           fill=False, edgecolor=color, linewidth=2)
        ax.add_patch(rect)
        
        # 트랙 ID 표시
        label = f"ID: {track.track_id}"
        if show_velocity:
            vx, vy = estimate_velocity(track)
            speed = np.sqrt(vx**2 + vy**2)
            label += f"\\nSpeed: {speed:.2f} m/s"
        
        ax.text(x1, y1-5, label, 
               color=color, fontsize=10, weight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.5))
        
        # 궤적 그리기
        if len(track.detections) > 1:
            centers = []
            for d in track.detections[-10:]:  # 최근 10개만
                cx = (d[0] + d[2]) / 2
                cy = (d[1] + d[3]) / 2
                centers.append([cx, cy])
            
            if len(centers) > 1:
                centers = np.array(centers)
                ax.plot(centers[:, 0], centers[:, 1], color=color, linewidth=2, alpha=0.5)
    
    plt.tight_layout()
    return fig

print("시각화 함수 정의 완료")

In [ ]:
# Cell 6: 비디오 시퀀스 추적 (예시)
# 여러 프레임에 걸친 추적 테스트
def track_video_sequence(tracker, image_dir, yolo_model, num_frames=10):
    """
    비디오 시퀀스에 대한 추적 수행
    """
    image_files = sorted(image_dir.glob("*.png"))[:num_frames]
    all_tracks = []
    
    for frame_idx, image_path in enumerate(image_files):
        # 이미지 로드
        image = cv2.imread(str(image_path))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # YOLO 탐지
        results = yolo_model(str(image_path))
        yolo_result = results[0]
        
        # 추적 업데이트
        tracks = track_objects(tracker, yolo_result, frame_idx)
        all_tracks.append((image_rgb, tracks))
        
        print(f"Frame {frame_idx+1}/{len(image_files)}: {len(tracks)} tracks")
    
    return all_tracks

# 사용 예시 (실제 실행 시 주석 해제)
# from ultralytics import YOLO
# yolo_model = YOLO('yolov8n.pt')
# train_images_dir = dataset_root / "images" / "train"
# tracked_sequence = track_video_sequence(tracker, train_images_dir, yolo_model, num_frames=10)